In [7]:
require(tidyverse)
require(caret)

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: caret

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [8]:
house_sales <- read.csv('house_sales.csv')

In [9]:
missing_city <- sum(str_count(house_sales$city, "--"))
missing_city

[1] 73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [10]:
house_sales <- read.csv('house_sales.csv')
colSums(is.na(house_sales)) #check for missing data
house_sales[house_sales == '--'] <- NA #replace missing values (--) with NA
colSums(is.na(house_sales)) #check for missing data

#Replace missing data and clean strings
house_sales$city <- house_sales$city %>% replace_na("Unknown")

house_sales$months_listed <- house_sales$months_listed %>% replace_na(round(mean(house_sales$months_listed, na.rm = TRUE), digits = 1))

house_sales$house_type <- str_replace(house_sales$house_type, "Det.", "Detached")
house_sales$house_type <- str_replace(house_sales$house_type, "Semi", "Semi-detached")	
house_sales$house_type <- str_replace(house_sales$house_type, "Detachedched", "Detached")
house_sales$house_type <- str_replace(house_sales$house_type, "Terr.", "Terraced")	
house_sales$house_type <- str_replace(house_sales$house_type, "Terracedced", "Terraced")
house_sales$house_type <- str_replace(house_sales$house_type, "Semi-detached-detached", "Semi-detached")

house_sales$area <- gsub("sq.m.", "", house_sales$area)

#Classify data types
house_sales$house_id <- factor(house_sales$house_id)
house_sales$city <- factor(house_sales$city)
house_sales$sale_date <- as.Date(house_sales$sale_date)
house_sales$sale_date <- ymd(house_sales$sale_date)
house_sales$sale_price <- as.integer(house_sales$sale_price)
house_sales$bedrooms <- as.integer(house_sales$bedrooms)
house_sales$house_type <- factor(house_sales$house_type)
house_sales$area <- round(as.numeric(house_sales$area), digits = 1)

clean_data <- house_sales 
clean_data

house_id          city    sale_price     sale_date months_listed 
            0             0             0             0            31 
     bedrooms    house_type          area 
            0             0             0

house_id          city    sale_price     sale_date months_listed 
            0            73             0             0            31 
     bedrooms    house_type          area 
            0             0             0

house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
<fct>,<fct>,<int>,<date>,<dbl>,<int>,<fct>,<dbl>
1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1
1539789,Riverford,47143,2020-09-25,5.1,2,Semi-detached,123.2
1353069,Teasdale,429914,2021-05-18,5.9,6,Detached,544.6
1155864,Teasdale,284440,2020-10-15,5.8,4,Detached,371.3
1608726,Teasdale,87134,2021-01-07,4.4,2,Semi-detached,125.7


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [11]:
price_by_rooms <- house_sales %>% group_by(bedrooms) %>% #group data
  summarise(avg_price = round(mean(sale_price), digits = 1),
         var_price = round(var(sale_price), digits = 1)) #calculate average and variance and round result
price_by_rooms

bedrooms,avg_price,var_price
<int>,<dbl>,<dbl>
2,67076.4,565289570
3,154665.1,2378289077
4,234704.6,1725211192
5,301515.9,2484327529
6,375741.3,3924432280


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [12]:
#Clean training and validation datasets

train <- read.csv('train.csv')
train$house_id <- factor(train$house_id)
train$city <- factor(train$city)
train$sale_date <- as.Date(train$sale_date)
train$sale_date <- ymd(train$sale_date)
train$sale_price <- as.integer(train$sale_price)
train$bedrooms <- as.integer(train$bedrooms)
train$house_type <- factor(train$house_type)
train$area <- round(as.numeric(train$area), digits = 1)

validation <- read.csv('validation.csv')
validation$house_id <- factor(validation$house_id)
validation$city <- factor(validation$city)
validation$sale_date <- as.Date(validation$sale_date)
validation$sale_date <- ymd(validation$sale_date)
validation$bedrooms <- as.integer(validation$bedrooms)
validation$house_type <- factor(validation$house_type)
validation$area <- round(as.numeric(validation$area), digits = 1)

In [13]:
#Baseline model: Linear model (model1)

set.seed(23)
model1 <- train(
  sale_price ~ bedrooms + area + house_type + city,
  train,
  method = "lm",
  trControl = trainControl(
    method = "cv",
    number = 10,
    verboseIter = TRUE
  )
)

#Predict on training data: p1
p1 <- predict(model1, train)

#Predict on new data and save as base_result:
base_result <- validation 
base_result$price <- predict(model1, newdata = base_result)

#Compute errors: error
error1 <- p1 - train[["sale_price"]]

#Calculate RMSE
sqrt(mean(error1 ^ 2)) 

base_result

+ Fold01: intercept=TRUE 
- Fold01: intercept=TRUE 
+ Fold02: intercept=TRUE 
- Fold02: intercept=TRUE 
+ Fold03: intercept=TRUE 
- Fold03: intercept=TRUE 
+ Fold04: intercept=TRUE 
- Fold04: intercept=TRUE 
+ Fold05: intercept=TRUE 
- Fold05: intercept=TRUE 
+ Fold06: intercept=TRUE 
- Fold06: intercept=TRUE 
+ Fold07: intercept=TRUE 
- Fold07: intercept=TRUE 
+ Fold08: intercept=TRUE 
- Fold08: intercept=TRUE 
+ Fold09: intercept=TRUE 
- Fold09: intercept=TRUE 
+ Fold10: intercept=TRUE 
- Fold10: intercept=TRUE 
Aggregating results
Fitting final model on full training set


[1] 21559.93

house_id,city,sale_date,months_listed,bedrooms,house_type,area,price
<fct>,<fct>,<date>,<dbl>,<int>,<fct>,<dbl>,<dbl>
1331375,Teasdale,2022-05-17,7.7,3,Terraced,209.7,120675.00
1630115,Teasdale,2020-06-30,6.5,4,Detached,390.6,303993.02
1645745,Silvertown,2020-09-02,7.4,6,Detached,556.8,383985.52
1336775,Silvertown,2021-10-03,8.8,3,Semi-detached,208.3,122480.57
1888274,Silvertown,2022-06-13,5.7,4,Detached,389.2,271257.60
1567679,Teasdale,2022-08-10,4.9,4,Detached,349.6,277705.36
1298770,Silvertown,2020-12-07,7.3,5,Detached,476.9,330122.09
1662892,Riverford,2020-07-26,5.3,2,Semi-detached,103.0,-12866.65
1514321,Poppleton,2020-03-02,4.5,3,Detached,291.3,170929.63


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [15]:
#Comparison model: Random forest model (model2)

set.seed(22)
model2 <- train(
  sale_price ~ bedrooms + area + house_type + city,
  tuneLength = 10,
  data = train, 
  method = "ranger",
  trControl = trainControl(
    method = "cv", 
    number = 5, 
    verboseIter = TRUE))

#Predict on training data
p2 <- predict(model2, train)

#Predict on new data and save as compare_result
compare_result <- validation
compare_result$price <- predict(model2, newdata = compare_result)

# Compute errors: error
error2 <- p2 - train[["sale_price"]]

# Calculate RMSE
sqrt(mean(error2 ^ 2)) 

compare_result

note: only 6 unique complexity parameters in default grid. Truncating the grid to 6 .

+ Fold1: mtry=2, min.node.size=5, splitrule=variance 
- Fold1: mtry=2, min.node.size=5, splitrule=variance 
+ Fold1: mtry=3, min.node.size=5, splitrule=variance 
- Fold1: mtry=3, min.node.size=5, splitrule=variance 
+ Fold1: mtry=4, min.node.size=5, splitrule=variance 
- Fold1: mtry=4, min.node.size=5, splitrule=variance 
+ Fold1: mtry=5, min.node.size=5, splitrule=variance 
- Fold1: mtry=5, min.node.size=5, splitrule=variance 
+ Fold1: mtry=6, min.node.size=5, splitrule=variance 
- Fold1: mtry=6, min.node.size=5, splitrule=variance 
+ Fold1: mtry=7, min.node.size=5, splitrule=variance 
- Fold1: mtry=7, min.node.size=5, splitrule=variance 
+ Fold1: mtry=2, min.node.size=5, splitrule=extratrees 
- Fold1: mtry=2, min.node.size=5, splitrule=extratrees 
+ Fold1: mtry=3, min.node.size=5, splitrule=extratrees 
- Fold1: mtry=3, min.node.size=5, splitrule=extratrees 
+ Fold1: mtry=4, min.node.size=5, splitru

[1] 11400.02

house_id,city,sale_date,months_listed,bedrooms,house_type,area,price
<fct>,<fct>,<date>,<dbl>,<int>,<fct>,<dbl>,<dbl>
1331375,Teasdale,2022-05-17,7.7,3,Terraced,209.7,82150.34
1630115,Teasdale,2020-06-30,6.5,4,Detached,390.6,301167.99
1645745,Silvertown,2020-09-02,7.4,6,Detached,556.8,396956.08
1336775,Silvertown,2021-10-03,8.8,3,Semi-detached,208.3,110617.10
1888274,Silvertown,2022-06-13,5.7,4,Detached,389.2,262310.27
1567679,Teasdale,2022-08-10,4.9,4,Detached,349.6,277608.71
1298770,Silvertown,2020-12-07,7.3,5,Detached,476.9,332190.54
1662892,Riverford,2020-07-26,5.3,2,Semi-detached,103.0,39522.02
1514321,Poppleton,2020-03-02,4.5,3,Detached,291.3,174299.44
